In [8]:
import pandas as pd
import numpy as np 
from numpy import array
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Conv2D,Embedding
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense,LSTM
from keras.layers import Activation, Dropout,BatchNormalization
from keras import optimizers
from keras import applications
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import math 
import datetime
import time
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import requests,io,random,re

In [9]:
r = requests.get("https://data.heatonresearch.com/data/t81-558/text/"\
                 "treasure_island.txt")
raw_text = r.text
print(raw_text[0:1000])

ï»¿The Project Gutenberg EBook of Treasure Island, by Robert Louis Stevenson

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: Treasure Island

Author: Robert Louis Stevenson

Illustrator: Milo Winter

Release Date: January 12, 2009 [EBook #27780]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK TREASURE ISLAND ***




Produced by Juliet Sutherland, Stephen Blundell and the
Online Distributed Proofreading Team at http://www.pgdp.net









 THE ILLUSTRATED CHILDREN'S LIBRARY


         _Treasure Island_

       Robert Louis Stevenson

          _Illustrated by_
            Milo Winter


           [Illustration]


           GRAMERCY BOOKS
              NEW YORK




 Foreword copyright Â© 1986 by Random House V


In [10]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text)

In [11]:
print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 397400
total chars: 60


In [12]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 132454


In [13]:
sentences

['the project gutenberg ebook of treasure ',
 ' project gutenberg ebook of treasure isl',
 'oject gutenberg ebook of treasure island',
 'ct gutenberg ebook of treasure island, b',
 'gutenberg ebook of treasure island, by r',
 'enberg ebook of treasure island, by robe',
 'erg ebook of treasure island, by robert ',
 ' ebook of treasure island, by robert lou',
 'ook of treasure island, by robert louis ',
 ' of treasure island, by robert louis ste',
 ' treasure island, by robert louis steven',
 'easure island, by robert louis stevenson',
 'ure island, by robert louis stevenson\r\n\r',
 ' island, by robert louis stevenson\r\n\r\nth',
 'land, by robert louis stevenson\r\n\r\nthis ',
 'd, by robert louis stevenson\r\n\r\nthis ebo',
 'by robert louis stevenson\r\n\r\nthis ebook ',
 'robert louis stevenson\r\n\r\nthis ebook is ',
 'ert louis stevenson\r\n\r\nthis ebook is for',
 ' louis stevenson\r\n\r\nthis ebook is for th',
 'uis stevenson\r\n\r\nthis ebook is for the u',
 ' stevenson\r\n\r\n

In [14]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [15]:
x.shape

(132454, 40, 60)

In [16]:
y.shape

(132454, 60)

In [17]:
y[0:10]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, Fal

In [20]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = 'rmsprop'
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [21]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               96768     
_________________________________________________________________
dense_2 (Dense)              (None, 60)                7740      
Total params: 104,508
Trainable params: 104,508
Non-trainable params: 0
_________________________________________________________________


In [22]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [44]:
import sys
def onepoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [45]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end= onepoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
132454/132454 [==============================] - 80s 604us/step - loss: 1.8803
****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "on, that's silver. a very
remarkable ma"
on, that's silver. a very
remarkable mane and the captain the captain the was and the come and the captain to the coptain of the captain the could of the captain the captain this the parter and the stare of the captain of the captain of the come the captain of the sime and and the come the store the captain the come of the captain of the captain the captain the coptain the was the store of the side and the come the was the sime and the
----- temperature: 0.5
----- Generating with seed: "on, that's silver. a very
remarkable ma"
on, that's silver. a very
remarkable mang the copping and with and and and the seppare beat the store and if the could of a mines the stow the can have whith the bectous


KeyboardInterrupt: 